# Chapter 7: Shortest Path

SSSP and APSP using the MIN_PLUS semiring.

In [ ]:
import graphblas as gb
from graphblas import Matrix, Vector, semiring, binary
import networkx as nx
import matplotlib.pyplot as plt
import math

## Weighted Graph

In [ ]:
edges = [(0,1,4), (0,2,2), (1,2,1), (1,3,5), (2,3,8), (2,4,10), (3,4,2)]
rows, cols, weights = zip(*edges)

A = Matrix.from_coo(rows, cols, weights, nrows=5, ncols=5, dtype=float)
print("Weighted adjacency matrix:")
print(A)

In [ ]:
G = nx.DiGraph()
for r, c, w in edges:
    G.add_edge(r, c, weight=w)

pos = {0: (0,1), 1: (1,1.5), 2: (1,0.5), 3: (2,1), 4: (3,1)}
nx.draw(G, pos, with_labels=True, node_color='lightblue', 
        node_size=500, font_size=16, arrows=True)
edge_labels = {(r,c): w for r,c,w in edges}
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
plt.title("Weighted Directed Graph")
plt.show()

## Single-Source Shortest Path (SSSP)

In [ ]:
def sssp(A, source):
    """Bellman-Ford style SSSP using MIN_PLUS semiring."""
    n = A.nrows
    
    # Initialize distances: 0 for source, absent for others
    dist = Vector(float, size=n)
    dist[source] = 0.0
    
    # Iterate until convergence
    for iteration in range(n - 1):
        # Relax edges: dist' = min(dist, dist × A)
        new_dist = dist.mxv(A, semiring.min_plus).new()
        new_dist = dist.ewise_add(new_dist, binary.min).new()
        
        if new_dist.isequal(dist):
            print(f"Converged after {iteration + 1} iterations")
            break
        dist = new_dist
    
    return dist

distances = sssp(A, source=0)
print("\nShortest distances from node 0:")
print(distances)

In [ ]:
# Verify with NetworkX
nx_distances = nx.single_source_dijkstra_path_length(G, 0)
print("NetworkX verification:")
for node, dist in sorted(nx_distances.items()):
    print(f"  Node {node}: {dist}")

## All-Pairs Shortest Path (APSP)

In [ ]:
def apsp(A):
    """Floyd-Warshall style APSP using MIN_PLUS semiring."""
    n = A.nrows
    
    # Initialize with adjacency matrix + zero diagonal
    D = A.dup()
    for i in range(n):
        D[i, i] = 0.0
    
    # Repeated matrix multiply until convergence
    for _ in range(int(math.log2(n)) + 1):
        D_new = D.mxm(D, semiring.min_plus).new()
        D = D.ewise_add(D_new, binary.min).new()
    
    return D

all_pairs = apsp(A)
print("All-pairs shortest distances:")
print(all_pairs)

In [ ]:
# Interpret the result
print("\nShortest paths:")
for i in range(5):
    for j in range(5):
        d = all_pairs.get(i, j)
        if d is not None and i != j:
            print(f"  {i} -> {j}: {d}")